<a href="https://colab.research.google.com/github/nephelim74/machinelearn/blob/main/ml_dz10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Шаг 1: Загрузка данных
Импортируем данные с помощью библиотеки pandas.
возьмём набор данных, например, "Wine Quality" (качество вина) из репозитория UCI Machine Learning. Этот набор данных содержит информацию о химических свойствах вин и их качестве (оценка от 0 до 10). Мы будем решать задачу регрессии, чтобы предсказать качество вина на основе его характеристик.


In [28]:
import pandas as pd

# Загрузка данных
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(url, delimiter=';')

# Посмотрим на данные
print(data.head())
print(data.info())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

Шаг 2: Подготовка данных
Разделим данные на признаки (X) и целевую переменную (y). Также разделим данные на обучающую и тестовую выборки.

In [17]:
from sklearn.model_selection import train_test_split

# Признаки и целевая переменная
X = data.drop('quality', axis=1)  # Все колонки, кроме 'quality'
y = data['quality']  # Целевая переменная 'quality'

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Шаг 3: Создание модели
Для задачи регрессии используем модель Gradient Boosting Regressor (градиентный бустинг).

In [18]:
from sklearn.ensemble import GradientBoostingRegressor

# Создание модели
model = GradientBoostingRegressor(random_state=42)

Шаг 4: Кросс-валидация
Используем метод cross_val_score для оценки производительности модели на обучающей выборке. В качестве метрики используем среднеквадратичную ошибку (MSE).

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

# Кросс-валидация с 5 фолдами
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring=mse_scorer)

# Вывод результатов кросс-валидации
print("Результаты кросс-валидации (MSE):", -cv_scores)
print("Средняя MSE:", -cv_scores.mean())

Результаты кросс-валидации (MSE): [0.46136001 0.40342771 0.40636128 0.43776887 0.31322647]
Средняя MSE: 0.40442886803667355


Средняя MSE: 0.404. Это хороший результат, но его можно улучшить, подобрав гиперпараметры.

Шаг 5: Подбор гиперпараметров с помощью GridSearchCV
Используем GridSearchCV для поиска оптимальных гиперпараметров.

In [20]:
from sklearn.model_selection import GridSearchCV

# Определение сетки гиперпараметров
param_grid = {
    'n_estimators': [100, 200, 300],  # Количество деревьев
    'learning_rate': [0.01, 0.1, 0.2],  # Скорость обучения
    'max_depth': [3, 5, 7]  # Максимальная глубина дерева
}

# Создание GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=mse_scorer)

# Поиск лучших гиперпараметров
grid_search.fit(X_train, y_train)

# Вывод лучших параметров
print("Лучшие параметры:", grid_search.best_params_)

Лучшие параметры: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 300}


Шаг 6: Обучение модели с оптимальными гиперпараметрами
Обучим модель с лучшими параметрами и оценим её производительность на тестовой выборке.

In [21]:
# Лучшая модель
best_model = grid_search.best_estimator_

# Оценка на тестовой выборке
y_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print("MSE на тестовой выборке:", test_mse)

MSE на тестовой выборке: 0.3598940844517581


## Выводы
Подбор гиперпараметров:

Использование GridSearchCV позволило найти оптимальные параметры, которые значительно улучшили качество модели.

Модель стала более точной и устойчивой.

Качество модели:

MSE на тестовой выборке (0.3599) близка к средней MSE на кросс-валидации (0.404), что говорит о хорошей обобщающей способности модели.

Модель не переобучилась, так как ошибка на тестовой выборке не увеличилась.